In [165]:
# -------- Code Outline -------- #
# Create a moving window that scrolls over the scatter plots
# This window will then flag AF if there is one AF sample within
# The window

In [166]:
import numpy as np
import matplotlib.pyplot as plt 
import pickle

In [167]:
# -------- Loader -------- #
# Load in the RR interval length, segmented data and the label array
# Which tells us if an RR interval is AF or not

# Load in the segments
with open('RR Interval Data/8455_RR_Interval_Length_Index.pkl', 'rb') as f:
    index = pickle.load(f)
    
with open('RR Interval Data/8455_RR_Interval_Length.pkl', 'rb') as f:
    interval_length = pickle.load(f)
    
# Load in the labels to tell us if that RR interval is AF or not
with open('RR Interval Data/8455_AF_truth_array.pkl', 'rb') as f:
    labels = pickle.load(f)

In [168]:
# -------- Normaliser -------- #
# Normalise all the RR interval data
# This makes it suitable for RNN use later on

mean = np.mean(interval_length)
interval_length -= mean
std = np.std(interval_length)
interval_length /= std

In [169]:
# -------- Window Strides -------- #
# We want windows of 80 samples wide traversing the scatter plot
# Then if there is at least one AF interval in the window, we set
# The window value to true

labels = np.array(labels)

# Create a window index number array
window_index = np.arange(1,(len(index) - 80) + 1, 1)
# Create a window size array (The number gives us the final x
# Value the window ends on)
window_width = np.arange(80, len(index), 1)
# Create an array to store window truth values
window_AF = np.array(window_index)

# Now we need to loop over all the labels and if there is a 
# 1 (label for AF) then we assign that window true (1)
# If not then we assign it false (0)

for c, i in enumerate(window_width):
    if (any(labels[(i - 80):(i - 1)]) == True):
        window_AF[c] = 1
    else:
        window_AF[c] = 0

In [170]:
# Now we need to extract each windows RR intervals
# Into sub arrays that we can feed into the RNN along with the label

window_arrays = []

for i,window_length in enumerate(window_width):
    window_arrays.append(interval_length[window_length - 80:window_length])

In [171]:
# -------- Saver -------- #
# Now that we have the window arrays and their labels we can
# Save these in a separate pickle file and use them for the RNN

# Export the window arrays

filename = '8455_Window_Arrays'
# This line uses pickle to save the window arrays as a .pkl file

with open('{}.pkl'.format(filename), 'wb') as f:
     pickle.dump(window_arrays, f)
        
# Now export the window labels

# filename_2 = '8455_Window_Labels'

# This line uses pickle to save the window labels as a .pkl file

# with open('{}.pkl'.format(filename_2), 'wb') as f:
#      pickle.dump(window_AF, f)